# Fake News Generator & Detector using GPT-2 and BERT

This project presents an interactive system that combines two powerful Natural Language Processing (NLP) models to address the problem of **fake news**:

- **Fake News Generation**: Using the GPT-2 language model, the system can generate synthetic news headlines based on a given prompt. This demonstrates how easily fake news can be created with modern generative AI.
  
- **Fake News Detection**: A fine-tuned BERT model is used to classify whether a given news headline is *real* or *fake*, helping showcase how AI can also be leveraged to combat misinformation.

The interface is built using **Gradio**, enabling an intuitive and accessible way to interact with both functionalities without writing any code.
>
---

## How to Use:
1. Run the notebook step-by-step from top to bottom.
2. The final cell will launch a simple interface where you can:
   - Generate fake news headlines.
   - Detect if a headline is real or fake.

## Step 1 : Install & Import Required Libraries

In [1]:
!pip install transformers datasets torch gradio --quiet

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
import gradio as gr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.5 MB/s eta 0:00:00


## Step 2 :  Loading Pretrained GPT-2 and BERT Models

In [2]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_model.config.pad_token_id = gpt2_tokenizer.eos_token_id

clf = pipeline("text-classification", model="jy46604790/Fake-News-Bert-Detect", tokenizer="jy46604790/Fake-News-Bert-Detect")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


## Step 3 : Defining Generation and Detection Functions

In [3]:
def generate_fake_news(prompt="Breaking News:"):
    input_ids = gpt2_tokenizer.encode(prompt, return_tensors='pt')
    attention_mask = torch.ones_like(input_ids)

    outputs = gpt2_model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=30,
        num_return_sequences=5,
        do_sample=True,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        pad_token_id=gpt2_tokenizer.eos_token_id
    )

    headlines = []
    for output in outputs:
        headline = gpt2_tokenizer.decode(output, skip_special_tokens=True)
        headlines.append(headline)
    return "\n".join([f"{i+1}. {h}" for i, h in enumerate(headlines)])


def detect_fake_news(text):
    result = clf(text[:500])[0]
    label = "FAKE" if result['label'] == "LABEL_0" else "REAL"
    confidence = round(result['score'] * 100, 2)
    return f"📝 Input: {text}\n Prediction: {label}\n Confidence: {confidence}%"

## Step 4 : Creating the Gradio Interface

In [6]:
def gradio_interface(task, text):
    if task == "Generate Fake News":
        return generate_fake_news(text if text else "Breaking News:")
    elif task == "Detect Real/Fake":
        return detect_fake_news(text)
    else:
        return "Invalid task."

interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Dropdown(["Generate Fake News", "Detect Real/Fake"], label="Choose Task"),
        gr.Textbox(lines=2, placeholder="Enter a prompt or headline...", label="Input Text")
    ],
    outputs="text",
    title="📰 Fake News Generator & Detector",
    description="Use GPT-2 to generate fake news headlines or BERT to detect whether a headline is real or fake."
)

## Step 5 : Launching the Interface

In [7]:
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6bc7968f31f6706222.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


![Fake News Generation](GeneratedDemo.png)

![Fake News Detection Demo](DetectDemo1.png)

![Fake News Detection Demo](DetectDemo2.png)